In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'

dir_raw = dir_data/'raw'
dir_ext = dir_data/'external'
dir_int = dir_data/'interim'
dir_out = dir_data/'processed'

gdb_raw = dir_raw/'raw.gdb'
gdb_int = dir_int/'interim.gdb'
gdb_out = dir_out/'processed.gdb'

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

In [4]:
import h3_arcgis

In [6]:
import h3.api.numpy_int as h3

In [8]:
aoi_fc = gdb_int/'cbsa_seattle'

In [9]:
aoi_df = pd.DataFrame.spatial.from_featureclass(aoi_fc)
aoi_df

,OBJECTID,CBSA_ID,NAME,SHAPE
0,1,42660,"Seattle-Tacoma-Bellevue, WA","{""rings"": [[[-122.63816200392569, 47.146658001..."


In [34]:
aoi_h3_ids = pd.Series(h3_arcgis.get_unique_h3_ids_for_aoi(aoi_df, 10))
aoi_h3_ids

0          622215106392391679
1          622217167903752191
2          622217167789129727
3          622217167570305023
4          622217167539044351
                  ...        
1158470    622214773979774975
1158471    622214735945957375
1158472    622214779148206079
1158473    622214769801265151
1158474    622214776647352319
Length: 1158475, dtype: int64

In [35]:
pt_df = pd.DataFrame.spatial.from_featureclass(gdb_int/'block_points')

In [36]:
pt_h3_ids = pt_df.SHAPE.apply(lambda pt: h3.geo_to_h3(pt.centroid[1], pt.centroid[0], 10))

In [37]:
aoi_h3_ids.isin(pt_h3_ids)

0           True
1          False
2          False
3          False
4          False
           ...  
1158470    False
1158471    False
1158472     True
1158473    False
1158474    False
Length: 1158475, dtype: bool

In [40]:
aoi_h3_df = aoi_h3_ids.to_frame(name='h3_l10')
aoi_h3_df['has_point'] = aoi_h3_ids.isin(pt_h3_ids)
aoi_h3_df.head()

,h3_l10,has_point
0,622215106392391679,True
1,622217167903752191,False
2,622217167789129727,False
3,622217167570305023,False
4,622217167539044351,False


In [51]:
aoi_h3_df['h3_l6'] = aoi_h3_df.h3_l10.apply(lambda h3_id: h3.h3_to_parent(h3_id, 6))
aoi_h3_df.head()

,h3_l10,has_point,h3_l9,h3_l6
0,622215106392391679,True,617711506765119487,604200707909746687
1,622217167903752191,False,617713568276545535,604202769494048767
2,622217167789129727,False,617713568161988607,604202769359831039
3,622217167570305023,False,617713567943098367,604202769091395583
4,622217167539044351,False,617713567911903231,604202769091395583


In [52]:
aoi_h3_df.join(aoi_h3_df.groupby('h3_l6')['has_point'].any(), on='h3_l6', rsuffix='_l6')

,h3_l10,has_point,h3_l9,h3_l6,has_point_l6
0,622215106392391679,True,617711506765119487,604200707909746687,True
1,622217167903752191,False,617713568276545535,604202769494048767,True
2,622217167789129727,False,617713568161988607,604202769359831039,False
3,622217167570305023,False,617713567943098367,604202769091395583,True
4,622217167539044351,False,617713567911903231,604202769091395583,True
...,...,...,...,...,...
1158470,622214773979774975,False,617711174352633855,604200375586652159,True
1158471,622214735945957375,False,617711136318685183,604200337468817407,False
1158472,622214779148206079,True,617711179521064959,604200380686925823,True
1158473,622214769801265151,False,617711170174058495,604200371425902591,True


In [58]:
aoi_h3_df_out = aoi_h3_df[aoi_h3_df.has_point]
aoi_h3_df_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41168 entries, 0 to 1158472
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   h3_l10     41168 non-null  int64
 1   has_point  41168 non-null  bool 
 2   h3_l9      41168 non-null  int64
 3   h3_l6      41168 non-null  int64
dtypes: bool(1), int64(3)
memory usage: 1.3 MB


In [60]:
h3_sedf = h3_arcgis.get_h3_hex_dataframe_from_h3_id_lst(aoi_h3_df_out.h3_l10)

In [63]:
h3_sedf.h3_id = h3_sedf.h3_id.astype(str)

In [64]:
h3_sedf.spatial.to_featureclass(gdb_int/'h3_l10_init')

'D:\\projects\\h3-arcgis\\data\\interim\\interim.gdb\\h3_l10_init'